In [1]:
import _pickle as pickle
import numpy as np

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [2]:
file_name = "data_batch_"
path = "/Users/zetong/cifar-10-batches-py/"
X = []
y = []
batch_size = 16
for i in range(1, 6):
    name = file_name + str(i)
    file_path = path + name
    dic = unpickle(file_path)
    X.append(dic[b'data'])
    y.append(dic[b'labels'])
X = np.concatenate(X)
y = np.concatenate(y)
X = X.reshape((X.shape[0], 3, 32, 32)).transpose(0, 2, 3, 1)
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x
X = normalize(X)
print(X.shape, y.shape)

(50000, 32, 32, 3) (50000,)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
X_train.shape

(29750, 32, 32, 3)

In [4]:
import tensorflow as tf
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).repeat()
test = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).repeat()
val = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).repeat()

In [5]:
i = tf.keras.layers.Input(shape=(32,32,3))
conv = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(i)
conv = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(conv)
max_pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
flat = tf.keras.layers.Flatten()(max_pool)
dense = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(flat)
out = tf.keras.layers.Dense(10, activation='softmax')(dense)
model = tf.keras.Model(i, out)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

W0908 01:47:35.190494 4661999040 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 1,050,890
Trainable params: 1,050,890
Non-trainable params: 0
___________________________________________________

In [7]:
stopping_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=20)
model.fit(train, callbacks=[stopping_early], validation_data=val, validation_steps=X_val.shape[0] / batch_size, steps_per_epoch=2000, epochs=100)

W0908 01:47:51.828736 4661999040 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/100
2000/2000 [==============================] - 49s 25ms/step - loss: 1.5008 - acc: 0.4683 - val_loss: 1.2711 - val_acc: 0.5402
Epoch 2/100
2000/2000 [==============================] - 18s 9ms/step - loss: 1.1755 - acc: 0.5909 - val_loss: 1.2064 - val_acc: 0.5714
Epoch 3/100
2000/2000 [==============================] - 17s 9ms/step - loss: 1.0460 - acc: 0.6359 - val_loss: 1.1600 - val_acc: 0.5960
Epoch 4/100
2000/2000 [==============================] - 16s 8ms/step - loss: 0.9408 - acc: 0.6760 - val_loss: 1.1970 - val_acc: 0.5865
Epoch 5/100
2000/2000 [==============================] - 16s 8ms/step - loss: 0.8507 - acc: 0.7076 - val_loss: 1.2259 - val_acc: 0.5874
Epoch 6/100
2000/2000 [==============================] - 16s 8ms/step - loss: 0.7735 - acc: 0.7344 - val_loss: 1.2301 - val_acc: 0.5912
Epoch 7/100
2000/2000 [==============================] - 16s 8ms/step - loss: 0.6995 - acc: 0.7607 - val_loss: 1.2713 - val_acc: 0.5990
Epoch 8/100
2000/2000 [========================

In [9]:
model.evaluate(test, steps=X_test.shape[0] / batch_size)

938/937 [==============================] - 15s 16ms/step - loss: 2.9624 - acc: 0.5611


[2.963961192639669, 0.5610667]

In [13]:
X = X.reshape((X.shape[0], 3, 32, 32)).transpose(0, 2, 3, 1)
print(X.shape)

(50000, 32, 32, 3)
